# Introduction to Reference Frames in Computer Vision

Simple example of a reference frame and a point in space.

In [1]:
import numpy as np
import plotly.graph_objects as go
import math

# Function to create a 2D reference frame with translation and rotation
def create_reference_frame_2d(fig, translation=[0, 0], rotation_degrees=0, name="", axis_length=1.0, show_labels=True):
    """
    Add a 2D reference frame to the plotly figure
    
    Args:
        fig: The plotly figure to add the frame to
        translation: [x, y] translation of the frame
        rotation_degrees: Rotation angle in degrees
        name: Name prefix for this frame (for the legend)
        axis_length: Length of the axes
        show_labels: Whether to show X and Y labels
        
    Returns:
        Updated figure
    """
    # Convert rotation to radians
    rotation_rad = math.radians(rotation_degrees)
    
    # Calculate the rotated X axis endpoint
    x_endpoint = [
        translation[0] + axis_length * math.cos(rotation_rad),
        translation[1] + axis_length * math.sin(rotation_rad)
    ]
    
    # Calculate the rotated Y axis endpoint
    y_endpoint = [
        translation[0] - axis_length * math.sin(rotation_rad),  # Negative because y-axis is 90° CCW from x-axis
        translation[1] + axis_length * math.cos(rotation_rad)
    ]
    
    # X axis with arrow
    fig.add_trace(go.Scatter(
        x=[translation[0], x_endpoint[0]], 
        y=[translation[1], x_endpoint[1]],
        mode='lines',
        line=dict(color='red', width=2),
        name=f'{name}X Axis'
    ))
    
    # Add arrow for X axis
    arrow_start_x = x_endpoint[0] - 0.2 * axis_length * math.cos(rotation_rad)
    arrow_start_y = x_endpoint[1] - 0.2 * axis_length * math.sin(rotation_rad)
    
    fig.add_annotation(
        x=x_endpoint[0], 
        y=x_endpoint[1],
        ax=arrow_start_x, 
        ay=arrow_start_y,
        xref="x", yref="y",
        axref="x", ayref="y",
        showarrow=True,
        arrowhead=2,
        arrowsize=1.5,
        arrowwidth=2,
        arrowcolor='red'
    )
    
    # Y axis with arrow
    fig.add_trace(go.Scatter(
        x=[translation[0], y_endpoint[0]], 
        y=[translation[1], y_endpoint[1]],
        mode='lines',
        line=dict(color='green', width=2),
        name=f'{name}Y Axis'
    ))
    
    # Add arrow for Y axis
    arrow_start_x = y_endpoint[0] - 0.2 * axis_length * (-math.sin(rotation_rad))
    arrow_start_y = y_endpoint[1] - 0.2 * axis_length * math.cos(rotation_rad)
    
    fig.add_annotation(
        x=y_endpoint[0], 
        y=y_endpoint[1],
        ax=arrow_start_x, 
        ay=arrow_start_y,
        xref="x", yref="y",
        axref="x", ayref="y",
        showarrow=True,
        arrowhead=2,
        arrowsize=1.5,
        arrowwidth=2,
        arrowcolor='green'
    )
    
    # Add labels if requested
    if show_labels:
        # X label
        label_offset_x = 0.05 * axis_length * math.sin(rotation_rad)
        label_offset_y = -0.05 * axis_length * math.cos(rotation_rad)
        
        fig.add_annotation(
            x=x_endpoint[0] + label_offset_x,
            y=x_endpoint[1] + label_offset_y,
            text="X",
            showarrow=False,
            font=dict(color='red', size=14),
            xref="x", yref="y"
        )
        
        # Y label
        label_offset_x = 0.05 * axis_length * math.cos(rotation_rad)
        label_offset_y = 0.05 * axis_length * math.sin(rotation_rad)
        
        fig.add_annotation(
            x=y_endpoint[0] + label_offset_x,
            y=y_endpoint[1] + label_offset_y,
            text="Y",
            showarrow=False,
            font=dict(color='green', size=14),
            xref="x", yref="y"
        )
    
    # Add origin point
    fig.add_trace(go.Scatter(
        x=[translation[0]], 
        y=[translation[1]],
        mode='markers',
        marker=dict(color='black', size=8, symbol='circle'),
        name=f'{name}Origin'
    ))
    
    return fig

# Function to add a point to the figure
def add_point_2d(fig, coords, name="P", color='blue', show_coordinates_on_frame=None):
    """
    Add a 2D point to the plotly figure
    
    Args:
        fig: The plotly figure to add the point to
        coords: [x, y] coordinates of the point
        name: Label for the point
        color: Color of the point
        show_coordinates_on_frame: If provided, show coordinates relative to this frame
                                 [tx, ty, rotation_degrees]
        
    Returns:
        Updated figure
    """
    # Add the point
    fig.add_trace(go.Scatter(
        x=[coords[0]], 
        y=[coords[1]],
        mode='markers+text',
        marker=dict(color=color, size=10),
        text=[name],
        textposition="top center",
        name=f"Point {name} ({coords[0]}, {coords[1]})"
    ))
    
    # If a reference frame is provided, show coordinates relative to that frame
    if show_coordinates_on_frame is not None:
        tx, ty = show_coordinates_on_frame[0], show_coordinates_on_frame[1]
        rotation_degrees = show_coordinates_on_frame[2]
        
        # Draw a dashed line from the frame origin to the point
        fig.add_trace(go.Scatter(
            x=[tx, coords[0]], 
            y=[ty, coords[1]],
            mode='lines',
            line=dict(color=color, width=1, dash='dash'),
            name=f'Connection to {name}'
        ))
        
        # Calculate the middle point of the line for placing coordinates
        mid_x = (tx + coords[0]) / 2
        mid_y = (ty + coords[1]) / 2
        
        # Calculate the coordinates in the frame's coordinate system
        # First translate to origin
        translated_x = coords[0] - tx
        translated_y = coords[1] - ty
        
        # Then rotate by the negative of the frame's rotation
        rotation_rad = math.radians(-rotation_degrees)
        rotated_x = translated_x * math.cos(rotation_rad) - translated_y * math.sin(rotation_rad)
        rotated_y = translated_x * math.sin(rotation_rad) + translated_y * math.cos(rotation_rad)
        
        # Round to 2 decimal places for display
        rotated_x = round(rotated_x, 2)
        rotated_y = round(rotated_y, 2)
        
        # Add the coordinates as an annotation directly on the line
        fig.add_annotation(
            x=mid_x,
            y=mid_y,
            text=f"({rotated_x}, {rotated_y})",
            showarrow=False,
            font=dict(color=color, size=12),
            bgcolor='rgba(255, 255, 255, 0.7)',
            bordercolor=color,
            borderwidth=1,
            borderpad=3
        )
    
    return fig

# Function to create a complete 2D visualization
def create_2d_visualization(points=None, frames=None, width=700, height=700, range_x=[-2, 2], range_y=[-2, 2]):
    """
    Create a complete 2D visualization with reference frames and points
    
    Args:
        points: List of points to add, each is [x, y, name, color, frame_index]
                frame_index is the index in frames to show coordinates on, or None
        frames: List of frames to add, each is [tx, ty, rotation_degrees, name]
        width: Width of the figure
        height: Height of the figure
        range_x: Range of x-axis to show
        range_y: Range of y-axis to show
        
    Returns:
        Plotly figure
    """
    # Create figure
    fig = go.Figure()
    
    # Default to empty lists if not provided
    if points is None:
        points = []
    if frames is None:
        frames = [[0, 0, 0, ""]]  # Default frame at origin
    
    # Add all frames
    for idx, frame in enumerate(frames):
        tx, ty, rotation_degrees, name = frame
        create_reference_frame_2d(fig, [tx, ty], rotation_degrees, name)
    
    # Add all points
    for point in points:
        x, y = point[0], point[1]
        name = point[2] if len(point) > 2 else "P"
        color = point[3] if len(point) > 3 else 'blue'
        
        # If a frame index is specified, get that frame for showing coordinates
        show_coords_frame = None
        if len(point) > 4 and point[4] is not None:
            frame_idx = point[4]
            if 0 <= frame_idx < len(frames):
                frame = frames[frame_idx]
                show_coords_frame = [frame[0], frame[1], frame[2]]
        
        add_point_2d(fig, [x, y], name, color, show_coords_frame)
    
    # Configure the figure
    fig.update_layout(
        title="2D Reference Frames and Points",
        xaxis=dict(range=range_x, title=""),
        yaxis=dict(range=range_y, title=""),
        width=width, height=height,
        plot_bgcolor='white',
        yaxis_scaleanchor="x",  # Ensures that X and Y scales are equal
        yaxis_scaleratio=1
    )
    
    # Grid
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', zeroline=True, zerolinewidth=2, zerolinecolor='gray')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', zeroline=True, zerolinewidth=2, zerolinecolor='gray')
    
    return fig

## Simple Example: Reference Frame and Point

In [2]:
import numpy as np
import plotly.graph_objects as go

# Create a 2D point
point = np.array([1, 1])

# Create a figure with Plotly
fig = go.Figure()

# Add the reference frame axes (origin) - with arrows at unit length
# X axis with arrow (red) - without text, we'll add it with an annotation
fig.add_trace(go.Scatter(
    x=[0, 1], y=[0, 0],
    mode='lines',  # Removed text from here
    line=dict(color='red', width=2),
    name='X Axis'
))

# Add arrow and label X for X axis
fig.add_annotation(
    x=1, y=0,
    ax=0.8, ay=0,
    xref="x", yref="y",
    axref="x", ayref="y",
    showarrow=True,
    arrowhead=2,
    arrowsize=1.5,
    arrowwidth=2,
    arrowcolor='red'
)

# Add X label as separate annotation for better positioning
fig.add_annotation(
    x=1.05,  # Positioned slightly after the axis end
    y=-0.05,  # Slightly below the axis
    text="X",
    showarrow=False,
    font=dict(color='red', size=14),
    xref="x", yref="y"
)

# Y axis with arrow (green) - without text
fig.add_trace(go.Scatter(
    x=[0, 0], y=[0, 1],
    mode='lines',  # Removed text from here
    line=dict(color='green', width=2),
    name='Y Axis'
))

# Add arrow for Y axis
fig.add_annotation(
    x=0, y=1,
    ax=0, ay=0.8,
    xref="x", yref="y",
    axref="x", ayref="y",
    showarrow=True,
    arrowhead=2,
    arrowsize=1.5,
    arrowwidth=2,
    arrowcolor='green'
)

# Add Y label as separate annotation for better positioning
fig.add_annotation(
    x=-0.05,  # Slightly to the left of the axis
    y=1.05,   # Positioned slightly above the axis end
    text="Y",
    showarrow=False,
    font=dict(color='green', size=14),
    xref="x", yref="y"
)

# Add the origin
fig.add_trace(go.Scatter(
    x=[0], y=[0],
    mode='markers',
    marker=dict(color='black', size=8, symbol='circle'),
    name='Origin'
))

# Add the point
fig.add_trace(go.Scatter(
    x=[point[0]], y=[point[1]],
    mode='markers+text',
    marker=dict(color='blue', size=10),
    text=['P'],
    textposition="top center",
    name=f"Point P ({point[0]}, {point[1]})"
))

# Calculate the middle point of the line for exact positioning
mid_x = (0 + point[0]) / 2
mid_y = (0 + point[1]) / 2

# Draw a dashed line from the origin to the point
fig.add_trace(go.Scatter(
    x=[0, point[0]], y=[0, point[1]],
    mode='lines',
    line=dict(color='blue', width=1, dash='dash'),
    name='Connection to frame'
))

# Add the coordinates as an annotation directly on the line
fig.add_annotation(
    x=mid_x,
    y=mid_y,
    text=f"({point[0]}, {point[1]})",
    showarrow=False,
    font=dict(color='blue', size=12),
    bgcolor='rgba(255, 255, 255, 0.7)',  # Semi-transparent white background
    bordercolor='blue',
    borderwidth=1,
    borderpad=3
)

# Add projections on the axes as dotted lines
# Projection on X axis - using annotation instead of text in scatter
fig.add_trace(go.Scatter(
    x=[0, point[0]], y=[0, 0],
    mode='lines',
    line=dict(color='blue', width=1, dash='dot'),
    name='X Projection'
))

# Add X projection label as annotation
fig.add_annotation(
    x=point[0]/2,
    y=-0.07,
    text=f"x={point[0]}",
    showarrow=False,
    font=dict(color='blue', size=12),
    xref="x", yref="y"
)

# Projection on Y axis - using annotation instead of text in scatter
fig.add_trace(go.Scatter(
    x=[0, 0], y=[0, point[1]],
    mode='lines',
    line=dict(color='blue', width=1, dash='dot'),
    name='Y Projection'
))

# Add Y projection label as annotation
fig.add_annotation(
    x=-0.07,
    y=point[1]/2,
    text=f"y={point[1]}",
    showarrow=False,
    font=dict(color='blue', size=12),
    xref="x", yref="y"
)

# Perpendicular lines from the point to the axes
fig.add_trace(go.Scatter(
    x=[point[0], point[0]], y=[0, point[1]],
    mode='lines',
    line=dict(color='gray', width=1, dash='dot'),
    name='Coordinates'
))

# Configure the figure
fig.update_layout(
    title="Reference Frame at Origin and Point P",
    xaxis=dict(range=[-0.1, 1.5], title=""),  # Adjusted range for better visualization
    yaxis=dict(range=[-0.1, 1.5], title=""),
    width=600, height=600,
    plot_bgcolor='white',
    yaxis_scaleanchor="x",  # Ensures that X and Y scales are equal
    yaxis_scaleratio=1
)

# Grid
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', zeroline=True, zerolinewidth=2, zerolinecolor='gray')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', zeroline=True, zerolinewidth=2, zerolinecolor='gray')

fig.show()

## Utility Functions for 3D Reference Frames Visualization

Below are functions to create and visualize 3D reference frames with different positions and orientations.

In [3]:
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation

def create_figure():
    """Create a new 3D figure with white background"""
    fig = go.Figure()
    
    # Set up the 3D axes
    fig.update_layout(
        scene=dict(
            xaxis=dict(range=[-1.5, 1.5], title=""),
            yaxis=dict(range=[-1.5, 1.5], title=""),
            zaxis=dict(range=[-1.5, 1.5], title=""),
            aspectmode='cube'
        ),
        width=700,
        height=700,
        title="3D Reference Frames",
        margin=dict(l=0, r=0, b=0, t=40),
        scene_camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
    )
    
    return fig

def rotation_matrix_from_euler(angles, sequence='xyz'):
    """Create a rotation matrix from Euler angles in degrees
    
    Args:
        angles (list): Euler angles in degrees [x, y, z]
        sequence (str): Rotation sequence, e.g., 'xyz', 'zyx'
        
    Returns:
        np.ndarray: 3x3 rotation matrix
    """
    rot = Rotation.from_euler(sequence, angles, degrees=True)
    return rot.as_matrix()

def add_reference_frame(fig, position=[0, 0, 0], rotation_matrix=None, scale=1.0, name="Frame", show_labels=True):
    """Add a reference frame to the figure at a specified position and orientation.
    
    Args:
        fig (go.Figure): Plotly figure to add the reference frame to
        position (list): Position of the reference frame origin [x, y, z]
        rotation_matrix (np.ndarray): 3x3 rotation matrix (None = identity)
        scale (float): Size of the coordinate axes
        name (str): Name for the reference frame
        show_labels (bool): Whether to show X, Y, Z labels
        
    Returns:
        go.Figure: Updated figure
    """
    position = np.array(position)
    
    if rotation_matrix is None:
        rotation_matrix = np.eye(3)  # Identity matrix
    
    # Create unit vectors for each axis
    x_axis = np.array([scale, 0, 0])  # X axis
    y_axis = np.array([0, scale, 0])  # Y axis
    z_axis = np.array([0, 0, scale])  # Z axis
    
    # Apply rotation
    x_axis_rotated = rotation_matrix @ x_axis
    y_axis_rotated = rotation_matrix @ y_axis
    z_axis_rotated = rotation_matrix @ z_axis
    
    # Define colors for axes
    x_color = 'red'
    y_color = 'green'
    z_color = 'blue'
    
    # Add origin
    fig.add_trace(go.Scatter3d(
        x=[position[0]],
        y=[position[1]],
        z=[position[2]],
        mode='markers',
        marker=dict(color='black', size=6),
        name=f"{name} Origin"
    ))
    
    # X-axis
    fig.add_trace(go.Scatter3d(
        x=[position[0], position[0] + x_axis_rotated[0]],
        y=[position[1], position[1] + x_axis_rotated[1]],
        z=[position[2], position[2] + x_axis_rotated[2]],
        mode='lines',
        line=dict(color=x_color, width=5),
        name=f"{name} X-axis"
    ))
    
    # Y-axis
    fig.add_trace(go.Scatter3d(
        x=[position[0], position[0] + y_axis_rotated[0]],
        y=[position[1], position[1] + y_axis_rotated[1]],
        z=[position[2], position[2] + y_axis_rotated[2]],
        mode='lines',
        line=dict(color=y_color, width=5),
        name=f"{name} Y-axis"
    ))
    
    # Z-axis
    fig.add_trace(go.Scatter3d(
        x=[position[0], position[0] + z_axis_rotated[0]],
        y=[position[1], position[1] + z_axis_rotated[1]],
        z=[position[2], position[2] + z_axis_rotated[2]],
        mode='lines',
        line=dict(color=z_color, width=5),
        name=f"{name} Z-axis"
    ))
    
    # Add labels if requested
    if show_labels:
        # X label
        fig.add_trace(go.Scatter3d(
            x=[position[0] + 1.1 * x_axis_rotated[0]],
            y=[position[1] + 1.1 * x_axis_rotated[1]],
            z=[position[2] + 1.1 * x_axis_rotated[2]],
            mode='text',
            text=['X'],
            textfont=dict(color=x_color, size=12),
            name=f"{name} X-label"
        ))
        
        # Y label
        fig.add_trace(go.Scatter3d(
            x=[position[0] + 1.1 * y_axis_rotated[0]],
            y=[position[1] + 1.1 * y_axis_rotated[1]],
            z=[position[2] + 1.1 * y_axis_rotated[2]],
            mode='text',
            text=['Y'],
            textfont=dict(color=y_color, size=12),
            name=f"{name} Y-label"
        ))
        
        # Z label
        fig.add_trace(go.Scatter3d(
            x=[position[0] + 1.1 * z_axis_rotated[0]],
            y=[position[1] + 1.1 * z_axis_rotated[1]],
            z=[position[2] + 1.1 * z_axis_rotated[2]],
            mode='text',
            text=['Z'],
            textfont=dict(color=z_color, size=12),
            name=f"{name} Z-label"
        ))
    
    return fig

def add_point(fig, position, color='blue', size=8, name=None, connect_to_origin=False, show_coordinates=False):
    """Add a 3D point to the figure.
    
    Args:
        fig (go.Figure): Plotly figure to add the point to
        position (list): Position of the point [x, y, z]
        color (str): Color of the point
        size (int): Size of the point marker
        name (str): Name for the point (defaults to P(x,y,z))
        connect_to_origin (bool): Whether to connect the point to the origin
        show_coordinates (bool): Whether to show the coordinates on the line
        
    Returns:
        go.Figure: Updated figure
    """
    position = np.array(position)
    
    if name is None:
        name = f"P({position[0]}, {position[1]}, {position[2]})"
    
    # Add the point
    fig.add_trace(go.Scatter3d(
        x=[position[0]],
        y=[position[1]],
        z=[position[2]],
        mode='markers+text',
        marker=dict(color=color, size=size, symbol='circle'),
        text=['P'],
        textposition="top center",
        name=name
    ))
    
    if connect_to_origin:
        # Connect to origin with a dashed line
        fig.add_trace(go.Scatter3d(
            x=[0, position[0]],
            y=[0, position[1]],
            z=[0, position[2]],
            mode='lines',
            line=dict(color=color, width=3, dash='dash'),
            name=f"Connection to origin"
        ))
        
        # Add coordinates label on the line if requested
        if show_coordinates:
            # Calculate midpoint of line
            mid_x = position[0] / 2
            mid_y = position[1] / 2
            mid_z = position[2] / 2
            
            fig.add_trace(go.Scatter3d(
                x=[mid_x],
                y=[mid_y],
                z=[mid_z],
                mode='text',
                text=[f"({position[0]}, {position[1]}, {position[2]})"],
                textfont=dict(color=color, size=10),
                name=f"Coordinates"
            ))
    
    return fig

def visualize_transform_with_point(translation=[0, 0, 0], rotation_angles=[0, 0, 0], point=[1, 1, 1], 
                                  show_coordinates=True, sequence='xyz'):
    """Create a 3D visualization with a reference frame at a given pose and a point.
    
    Args:
        translation (list): Translation of the reference frame [x, y, z]
        rotation_angles (list): Euler angles in degrees [x, y, z]
        point (list): Point coordinates in the world frame [x, y, z]
        show_coordinates (bool): Whether to show coordinates on the line connecting to the point
        sequence (str): Rotation sequence for Euler angles
        
    Returns:
        go.Figure: Figure with the reference frame and point
    """
    fig = create_figure()
    
    # Add world reference frame at origin
    fig = add_reference_frame(fig, position=[0, 0, 0], name="World")
    
    # Create rotation matrix from Euler angles
    R = rotation_matrix_from_euler(rotation_angles, sequence)
    
    # Add transformed reference frame
    if np.any(translation) or np.any(rotation_angles):
        fig = add_reference_frame(fig, position=translation, rotation_matrix=R, name="Transformed")
    
    # Add the point
    fig = add_point(fig, point, connect_to_origin=True, show_coordinates=show_coordinates)
    
    return fig

In [4]:
import numpy as np
import plotly.graph_objects as go

# Create a 2D point
point = np.array([1, 1])

# Create a figure with Plotly
fig = go.Figure()

# Add the reference frame axes (origin) - with arrows at unit length
# X axis with arrow (red) - without text, we'll add it with an annotation
fig.add_trace(go.Scatter(
    x=[0, 1], y=[0, 0],
    mode='lines',  # Removed text from here
    line=dict(color='red', width=2),
    name='X Axis'
))

# Add arrow and label X for X axis
fig.add_annotation(
    x=1, y=0,
    ax=0.8, ay=0,
    xref="x", yref="y",
    axref="x", ayref="y",
    showarrow=True,
    arrowhead=2,
    arrowsize=1.5,
    arrowwidth=2,
    arrowcolor='red'
)

# Add X label as separate annotation for better positioning
fig.add_annotation(
    x=1.05,  # Positioned slightly after the axis end
    y=-0.05,  # Slightly below the axis
    text="X",
    showarrow=False,
    font=dict(color='red', size=14),
    xref="x", yref="y"
)

# Y axis with arrow (green) - without text
fig.add_trace(go.Scatter(
    x=[0, 0], y=[0, 1],
    mode='lines',  # Removed text from here
    line=dict(color='green', width=2),
    name='Y Axis'
))

# Add arrow for Y axis
fig.add_annotation(
    x=0, y=1,
    ax=0, ay=0.8,
    xref="x", yref="y",
    axref="x", ayref="y",
    showarrow=True,
    arrowhead=2,
    arrowsize=1.5,
    arrowwidth=2,
    arrowcolor='green'
)

# Add Y label as separate annotation for better positioning
fig.add_annotation(
    x=-0.05,  # Slightly to the left of the axis
    y=1.05,   # Positioned slightly above the axis end
    text="Y",
    showarrow=False,
    font=dict(color='green', size=14),
    xref="x", yref="y"
)

# Add the origin
fig.add_trace(go.Scatter(
    x=[0], y=[0],
    mode='markers',
    marker=dict(color='black', size=8, symbol='circle'),
    name='Origin'
))

# Add the point
fig.add_trace(go.Scatter(
    x=[point[0]], y=[point[1]],
    mode='markers+text',
    marker=dict(color='blue', size=10),
    text=['P'],
    textposition="top center",
    name=f"Point P ({point[0]}, {point[1]})"
))

# Calculate the middle point of the line for exact positioning
mid_x = (0 + point[0]) / 2
mid_y = (0 + point[1]) / 2

# Draw a dashed line from the origin to the point
fig.add_trace(go.Scatter(
    x=[0, point[0]], y=[0, point[1]],
    mode='lines',
    line=dict(color='blue', width=1, dash='dash'),
    name='Connection to frame'
))

# Add the coordinates as an annotation directly on the line
fig.add_annotation(
    x=mid_x,
    y=mid_y,
    text=f"({point[0]}, {point[1]})",
    showarrow=False,
    font=dict(color='blue', size=12),
    bgcolor='rgba(255, 255, 255, 0.7)',  # Semi-transparent white background
    bordercolor='blue',
    borderwidth=1,
    borderpad=3
)

# Add projections on the axes as dotted lines
# Projection on X axis - using annotation instead of text in scatter
fig.add_trace(go.Scatter(
    x=[0, point[0]], y=[0, 0],
    mode='lines',
    line=dict(color='blue', width=1, dash='dot'),
    name='X Projection'
))

# Add X projection label as annotation
fig.add_annotation(
    x=point[0]/2,
    y=-0.07,
    text=f"x={point[0]}",
    showarrow=False,
    font=dict(color='blue', size=12),
    xref="x", yref="y"
)

# Projection on Y axis - using annotation instead of text in scatter
fig.add_trace(go.Scatter(
    x=[0, 0], y=[0, point[1]],
    mode='lines',
    line=dict(color='blue', width=1, dash='dot'),
    name='Y Projection'
))

# Add Y projection label as annotation
fig.add_annotation(
    x=-0.07,
    y=point[1]/2,
    text=f"y={point[1]}",
    showarrow=False,
    font=dict(color='blue', size=12),
    xref="x", yref="y"
)

# Perpendicular lines from the point to the axes
fig.add_trace(go.Scatter(
    x=[point[0], point[0]], y=[0, point[1]],
    mode='lines',
    line=dict(color='gray', width=1, dash='dot'),
    name='Coordinates'
))

# Configure the figure
fig.update_layout(
    title="Reference Frame at Origin and Point P",
    xaxis=dict(range=[-0.1, 1.5], title=""),  # Adjusted range for better visualization
    yaxis=dict(range=[-0.1, 1.5], title=""),
    width=600, height=600,
    plot_bgcolor='white',
    yaxis_scaleanchor="x",  # Ensures that X and Y scales are equal
    yaxis_scaleratio=1
)

# Grid
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', zeroline=True, zerolinewidth=2, zerolinecolor='gray')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', zeroline=True, zerolinewidth=2, zerolinecolor='gray')

fig.show()

## Examples Using the New Functions

In [5]:
# Example 1: Simple reference frame at origin with a point
fig = create_2d_visualization(
    points=[[1, 1, "P", "blue", 0]],  # Point at (1,1) showing coords relative to frame 0
    frames=[[0, 0, 0, ""]]  # Frame at origin with 0 rotation
)

fig.show()

In [6]:
# Example 2: Two frames - one at origin, one translated and rotated, with a point
fig = create_2d_visualization(
    points=[[1.5, 1, "P", "blue", 1]],  # Point showing coords relative to frame 1
    frames=[
        [0, 0, 0, ""],                # Frame at origin
        [0.5, 0, 30, "Rotated "]      # Frame at (0.5, 0) rotated 30 degrees
    ]
)

fig.show()

In [7]:
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objects as go
from IPython.display import clear_output

# Create widgets for user input
x_input = widgets.FloatText(
    value=0.5,
    description='X:',
    disabled=False,
    step=0.1
)

y_input = widgets.FloatText(
    value=0.0,
    description='Y:',
    disabled=False,
    step=0.1
)

rotation_slider = widgets.IntSlider(
    value=30,
    min=-180,
    max=180,
    step=5,
    description='Rotation:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

output_area = widgets.Output()

# Create function to update the visualization based on widget values
def update_visualization(x, y, rotation):
    with output_area:
        clear_output(wait=True)
        fig = create_2d_visualization(
            points=[[1.5, 1, "P", "blue", 1]],  # Point showing coords relative to frame 1
            frames=[
                [0, 0, 0, ""],                 # Frame at origin
                [x, y, rotation, "Rotated "]   # Frame with user-defined translation and rotation
            ]
        )
        fig.show()

# Function to handle widget changes
def on_value_change(change):
    update_visualization(x_input.value, y_input.value, rotation_slider.value)

# Register callbacks
x_input.observe(on_value_change, names='value')
y_input.observe(on_value_change, names='value')
rotation_slider.observe(on_value_change, names='value')

# Create UI layout
controls = widgets.HBox([x_input, y_input, rotation_slider])
ui = widgets.VBox([controls, output_area])

# Display the UI
display(ui)

# Show initial visualization
update_visualization(x_input.value, y_input.value, rotation_slider.value)